# Train and evaluate single-scale baseline #1
In this notebook, the single-scale ResNet18-U-Net used as baseline by Schmitz et al. (2021) is trained and evaluated on terminal H&E data for the purpose of providing a benchmark for the multi-scale model.

Training and evaluation are done in 5-fold cross-validation using the same folds as for the other competing models.

## Train and evaluate model

In [ ]:
import constants as c
import crossvalidation as cv
from data import MultiscaleDataset, NumpyDatasetSource
from data_collection import DataCollection, DataCollectionConfig
from data_preparation import DataPreparation, DataPreparationConfig
from data_sampling import StratifyingDataSampling
from experiment import Experiment
from prediction import Prediction, PredictionConfig
from singlescale.schmitz2021 import ResNetUNet
from training import Training, TrainingConfig


terminal_sample_dirs = cv.get_terminal_sample_dirs()
terminal_targets_dir = cv.get_terminal_targets_dir()
cv_folds = cv.get_folds()


for i, (
    train_images,
    val_images,
    terminal_test_images,
    _,
) in enumerate(cv_folds):
    with Experiment(name=f"singlescale1_run_4_cv_fold_{i}", seed=c.seed) as exp:
        terminal_data_collection = DataCollection(
            exp.working_dir,
            DataCollectionConfig(
                sample_dirs=[terminal_sample_dirs[0]],
                targets_dir=terminal_targets_dir,
                train_images=train_images,
                val_images=val_images,
                test_images=terminal_test_images,
            ),
            exp.dry_run,
            name="terminal_data_collection",
        )
        (
            train_entries,
            val_entries,
            terminal_test_entries,
        ) = terminal_data_collection.run()

        # Train model on terminal H&Es:

        data_sampling = StratifyingDataSampling(exp.working_dir, exp.dry_run)
        train_strata, val_strata = data_sampling.run(train_entries, val_entries)

        data_preparation = DataPreparation(
            exp.working_dir,
            DataPreparationConfig(
                resampling_seed=1914177978,
            ),
            exp.dry_run,
        )
        train_data, val_data = data_preparation.run(
            train_strata,
            val_strata,
        )

        model = ResNetUNet()

        training = Training(
            exp.working_dir,
            TrainingConfig(
                train_batch_size=24,
                val_batch_size=24,
                epochs=120,
                early_stopping=True,
                early_stopping_patience=30,
            ),
            exp.dry_run,
        )
        best_model = training.run(model, train_data, val_data)
        del model

        # Test model on terminal H&Es:

        terminal_test_data = {
            image_name: MultiscaleDataset(NumpyDatasetSource(entries))
            for image_name, entries in terminal_test_entries.items()
        }
        terminal_prediction = Prediction(
            exp.working_dir,
            PredictionConfig(
                batch_size=24,
                save_pred=False,
            ),
            exp.dry_run,
        )
        terminal_prediction.run(best_model, terminal_test_data, pred_subdir="terminal")

        del best_model
